In [1]:
from IPython.core.debugger import set_trace

%load_ext nb_black

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

%matplotlib inline

<IPython.core.display.Javascript object>

In [2]:
df_nse = pd.read_csv("./stock_data.csv")

df_nse["Date"] = pd.to_datetime(df_nse.Date, format="%Y-%m-%d")
df_nse.index = df_nse["Date"]
data = df_nse.sort_index(ascending=True, axis=0)

<IPython.core.display.Javascript object>

In [3]:
df = data

<IPython.core.display.Javascript object>

In [4]:
df.head(5)

,Date,Open,High,Low,Close,Volume,OpenInt,Stock
Date,,,,,,,,
1984-09-07,1984-09-07,0.42388,0.42902,0.41874,0.42388,23220030,0,AAPL
1984-09-10,1984-09-10,0.42388,0.42516,0.41366,0.42134,18022532,0,AAPL
1984-09-11,1984-09-11,0.42516,0.43668,0.42516,0.42902,42498199,0,AAPL
1984-09-12,1984-09-12,0.42902,0.43157,0.41618,0.41618,37125801,0,AAPL
1984-09-13,1984-09-13,0.43927,0.44052,0.43927,0.43927,57822062,0,AAPL


<IPython.core.display.Javascript object>

In [5]:
df = df[["Close"]].copy()

<IPython.core.display.Javascript object>

In [6]:
df.head(5)

,Close
Date,
1984-09-07,0.42388
1984-09-10,0.42134
1984-09-11,0.42902
1984-09-12,0.41618
1984-09-13,0.43927


<IPython.core.display.Javascript object>

In [7]:
def rate_of_change(df, n):
    M = df["Close"].diff(n - 1)
    N = df["Close"].shift(n - 1)
    ROC = pd.Series(M / N, name="ROC_" + str(n))
    df = df.join(ROC)
    return df


df = rate_of_change(df, 3)

<IPython.core.display.Javascript object>

In [8]:
df

,Close,ROC_3
Date,,
1984-09-07,0.42388,NaN
1984-09-10,0.42134,NaN
1984-09-11,0.42902,0.012126
1984-09-12,0.41618,-0.012247
1984-09-13,0.43927,0.023892
...,...,...
2017-11-10,83.87000,-0.411004
2017-11-10,178.46000,1.077179
2017-11-10,178.46000,0.728902


<IPython.core.display.Javascript object>

In [9]:
datatrain = df[["ROC_3"]].copy()

<IPython.core.display.Javascript object>

In [10]:
datatrain.head(5)

,ROC_3
Date,
1984-09-07,NaN
1984-09-10,NaN
1984-09-11,0.012126
1984-09-12,-0.012247
1984-09-13,0.023892


<IPython.core.display.Javascript object>

In [11]:
datatrain["ROC_3_Target"] = df.ROC_3.shift(-1)

<IPython.core.display.Javascript object>

In [12]:
datatrain

,ROC_3,ROC_3_Target
Date,,
1984-09-07,NaN,NaN
1984-09-10,NaN,0.012126
1984-09-11,0.012126,-0.012247
1984-09-12,-0.012247,0.023892
1984-09-13,0.023892,0.070835
...,...,...
2017-11-10,-0.411004,1.077179
2017-11-10,1.077179,0.728902
2017-11-10,0.728902,-0.519837


<IPython.core.display.Javascript object>

In [13]:
def train_test_split(data, percent):
    data = data.values
    n = int(len(data) * (1 - percent))
    return data[:n], data[n:]

<IPython.core.display.Javascript object>

In [14]:
train, test = train_test_split(datatrain, 0.1)

<IPython.core.display.Javascript object>

In [15]:
print(len(df))
print(len(train))
print(len(test))

51266
46139
5127


<IPython.core.display.Javascript object>

In [16]:
X = train[:, :-1]
y = train[:, -1]

<IPython.core.display.Javascript object>

In [17]:
from xgboost import XGBRegressor

model = XGBRegressor(objective="reg:squarederror", n_estimators=1000)
model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

<IPython.core.display.Javascript object>

In [18]:
model.save_model("roc_xgboost_model.txt")

<IPython.core.display.Javascript object>

In [19]:
predictions = []
for i in range(len(test)):
    inputVal = test[i, 0]

    val = np.array(inputVal).reshape(1, -1)
    pre = model.predict(val)
    predictions.append(pre[0])

<IPython.core.display.Javascript object>

In [20]:

def train_test_split_plotdata(data, percent):
    n = int(len(data) * (1 - percent))
    return data[n:]



<IPython.core.display.Javascript object>

In [21]:
valid = train_test_split_plotdata(df, 0.1)
valid["Predictions"] = predictions

<ipython-input-21-a4e38d5024cf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid["Predictions"] = predictions


<IPython.core.display.Javascript object>

In [26]:
test = train_test_split_plotdata(datatrain, 0.1)
test

,ROC_3,ROC_3_Target
Date,,
2016-08-05,-0.457461,-0.455941
2016-08-05,-0.455941,0.870922
2016-08-05,0.870922,0.849711
2016-08-05,0.849711,-0.457461
2016-08-05,-0.457461,-0.455941
...,...,...
2017-11-10,-0.411004,1.077179
2017-11-10,1.077179,0.728902
2017-11-10,0.728902,-0.519837


<IPython.core.display.Javascript object>

In [28]:
valid["target"] = test.ROC_3_Target


<ipython-input-28-a9aabed79d85>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid["target"] = test.ROC_3_Target


<IPython.core.display.Javascript object>

In [29]:
valid

,Close,ROC_3,Predictions,target
Date,,,,
2016-08-05,125.15,-0.457461,0.215633,-0.455941
2016-08-05,125.15,-0.455941,0.523565,0.870922
2016-08-05,125.15,0.870922,0.171293,0.849711
2016-08-05,105.23,0.849711,-0.077342,-0.457461
2016-08-05,105.23,-0.457461,0.215633,-0.455941
...,...,...,...,...
2017-11-10,83.87,-0.411004,0.528718,1.077179
2017-11-10,178.46,1.077179,-0.138400,0.728902
2017-11-10,178.46,0.728902,0.166606,-0.519837


<IPython.core.display.Javascript object>